# DA Algorithm
### 中田 竜明

In [128]:
include("da.jl")

DA

In [129]:
#;cat "da.jl"

In [142]:
m, n = 100, 100
m_prefs, f_prefs = DA.generate_random_preference_data(m, n)

(
101x100 Array{Int64,2}:
 59  98  100  91   73   59  100   18  …   0  91  82    2  56  70  28   0  34
 16  74   36   8   13    8   17   30     51  33  23   69  65  26  15  90  42
 20  47   59  88   37   80   62   84     94  16   9   66  77  10  71  82  20
  1  88   78  66   34   95   14   17     87  82  53    6  20  98  26   5  59
 47  38   56  93   56   55   43   55     43  66  28   75  85   7  72  37  22
 15  76   20  18   43   45   68   89  …  44  31  81   44  52   4  20  48  88
 77  79   17  19   23   90   41   92     28  63  73   23  93  17  66  35  66
 26  72   68  81   57    9   36   43     84   4  34   88  14  44  51  52   1
 30  35    8  15   71   46   25    6     31  32  89   76  72  15  80  85  25
 24  59   81   5   36   44    3   45     13  51   4    0  99  57  41  14  86
 13  57   82  59    6   81   64  100  …  30  19  39   48  27  65  43  71  18
  8  56   14  55   96   36   63    9     85  84  79   16  81  25  77  95  16
 80  60   13  70  100   35   35   59      1  71  5

In [143]:
DA.call_match(m, n, m_prefs, f_prefs)
DA.check_data(m, n, m_prefs, f_prefs)

true

In [140]:
m_f, f_m = DA.call_match(m, n, m_prefs, f_prefs)

([0,0,0,0,69,0,0,0,0,72  …  0,0,0,0,0,0,65,88,0,0],[43,0,0,0,16,0,54,70,0,70  …  25,44,0,0,0,0,15,0,0,0])

In [124]:
DA.check_results(m_f, f_m)

LoadError: LoadError: Matching Incomplete with female 14, f_m[14] = 34 though m_f[34] = 2
while loading In[124], in expression starting on line 1

In [125]:
f_m

100-element Array{Int64,1}:
  0
 34
  3
  0
  6
  0
  0
  0
  0
  0
 94
 17
  0
  ⋮
  0
  0
  0
 71
  0
  0
  0
 55
 15
  0
 80
  0

In [126]:
f_m[8]

0

In [127]:
m_f[35]

25

In [157]:
m, n = 10, 10
for i in 1:100
    m_prefs, f_prefs = DA.generate_random_preference_data(m, n)
    DA.call_match(m, n, m_prefs, f_prefs)
    try
        DA.check_data(m, n, m_prefs, f_prefs)
    catch
        println(m_prefs, f_prefs)
        break
    end
end